In [3]:
%pip install h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import os, sys
from tqdm.notebook import tqdm
sys.path.append(r"/home/rapids/notebooks/lisbet_code")
from utils.data import load_embedding


In [8]:
data_r = r"/home/rapids/notebooks/bet_embedders/13879972"
seed = 42

In [ ]:

datapath = os.path.join(data_r, "embedding_train.numpy")
dataval = os.path.join(data_r, "embedding_test.numpy")

# Load your temporal encoded data (shape: n_samples, 128 dimensions)
train_data =  load_embedding(datapath)# Replace with your data

In [ ]:
td = train_data.drop(columns='video').to_numpy()

In [ ]:
# Step 1: Dimensionality Reduction using PCA
pca = PCA(n_components=2)  # Reduce to 2 dimensions for visualization
data_2d = pca.fit_transform(td)


In [ ]:
data_2d.shape

In [6]:
# Step 2: K-Means Clustering and Finding the Optimal Number of Clusters
def kmeans_clustering(data, max_clusters=10, min_clusters=2, step=1):
    inertia = []
    silhouette = []

    for k in tqdm(range(min_clusters, max_clusters + 1, step)):
        kmeans = KMeans(n_clusters=k, random_state=seed)
        labels = kmeans.fit_predict(data)
        inertia.append(kmeans.inertia_)
        silhouette.append(silhouette_score(data, labels))
    return inertia, silhouette

# Run clustering for up to 10 clusters
min_clusters = 5
max_clusters = 100
step=5
inertia, silhouette = kmeans_clustering(data_2d, max_clusters, min_clusters, step)

  0%|          | 0/20 [00:00<?, ?it/s]

c:\Users\chataint\AppData\Local\anaconda3\envs\diffversify\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\chataint\AppData\Local\anaconda3\envs\diffversify\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

In [ ]:
# Step 3: Plot Inertia and Silhouette Scores
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(range(min_clusters, max_clusters + 1, step), inertia, marker='o')
ax[0].set_title("Inertia (Elbow Method)")
ax[0].set_xlabel("Number of Clusters")
ax[0].set_ylabel("Inertia")

ax[1].plot(range(min_clusters, max_clusters + 1, step), silhouette, marker='o')
ax[1].set_title("Silhouette Score")
ax[1].set_xlabel("Number of Clusters")
ax[1].set_ylabel("Silhouette Score")

plt.show()

c:\Users\chataint\AppData\Local\anaconda3\envs\diffversify\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

In [ ]:
# Step 4: KMeans Visualization (2D data)
optimal_clusters = silhouette.index(max(silhouette)) + 2  # Choose the optimal cluster
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
labels = kmeans.fit_predict(data_2d)

plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='viridis', s=10)
plt.title(f"KMeans Clustering with {optimal_clusters} Clusters")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(label="Cluster Label")
plt.show()

(55, 2)

# spectral study

In [ ]:
# Step 1: Compute the Covariance Matrix
cov_matrix = np.cov(td.T)  # Transpose data to have features as rows
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Step 2: Plot Eigenvalues
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(eigenvalues) + 1), sorted(eigenvalues, reverse=True), marker='o')
plt.title("Eigenvalues of Covariance Matrix")
plt.xlabel("Index")
plt.ylabel("Eigenvalue")
plt.show()



In [ ]:
# Step 3: Spectral Clustering Based on Dominant Eigenvalues
# Select the top-k eigenvalues for clustering
top_k = 5  # Adjust based on the elbow point in eigenvalues plot
spectral = SpectralClustering(n_clusters=top_k, affinity='nearest_neighbors', random_state=42)
labels = spectral.fit_predict(td)

# Step 4: Evaluate and Visualize Clustering
silhouette_avg = silhouette_score(td, labels)
print(f"Silhouette Score for {top_k} clusters: {silhouette_avg:.3f}")

# Visualization of clustering in PCA-reduced 2D space
plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='plasma', s=10)
plt.title(f"Spectral Clustering with {top_k} Clusters")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(label="Cluster Label")
plt.show()